In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import base64
import datetime

In [2]:
db_params = {
    "dbname": "rewrapped",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost", 
    "port": 5432          
}
connection_string = "postgresql://{user}:{password}@{host}:{port}/{dbname}".format(**db_params)
engine = create_engine(connection_string)

In [3]:
users = pd.read_sql("SELECT * FROM users", engine)
users['access_token'] = None
users

,id,email,name,refresh_token,hash_code,access_token
0,3,khaledmromeh@gmail.com,None,AQBgw9bmnN6QbcRT7J1SZnqkmeI4oLjnT0Aa_GqULz2gh_...,199b8261983dc42ff0047ba918aad9d6b7ddee8d2e12a1...,None


In [4]:
refresh_token = users.loc[0,'refresh_token']
client_id = "6a6355fbeb044695930d74e002d91214"
client_secret = "2722c709f97248889ba35a4f33069ced"

auth_string = f"{client_id}:{client_secret}"
auth_bytes = auth_string.encode('utf-8')
auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_base64}"
}
for i, user in users.iterrows():
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        response_json = response.json()
        users.loc[0,'access_token'] = response_json['access_token']
    else:
        print(f"Error: {response.content}")

In [45]:
top_artists = pd.read_sql("SELECT * FROM user_top_artists", engine)
top_artists

,artist_id,month,user_id,year,artist_name,rank


In [46]:
url = "https://api.spotify.com/v1/me/top/artists?time_range=short_term"
date = datetime.datetime.now()
year = date.strftime("%Y")
month = date.strftime("%m")
for i, user in users.iterrows():
    headers = {
        "Authorization": f"Bearer {users.loc[0,'access_token']}"
    }
    response = requests.get(url, headers=headers)
    artists =response.json()['items']
    for rank, artist in enumerate(artists):
        record = {
            "rank": rank+1,
            "user_id": user['id'],
            "year": year,
            "month": month,
            "artist_id": artist['id'],
            "artist_name": artist['name'],
        }
        record_df = pd.DataFrame(record, index=[0])
        top_artists = pd.concat([top_artists,record_df])
top_artists = top_artists.drop_duplicates(['user_id', 'artist_id', 'year', 'month'])
top_artists

,artist_id,month,user_id,year,artist_name,rank
0,3eVa5w3URK5duf6eyVDbu9,01,3,2025,ROSÉ,1
0,57nPqD7z62gDdq37US9XJR,01,3,2025,Arcane,2
0,4tuJ0bMpJh08umKkEXKUI5,01,3,2025,Gracie Abrams,3
0,06HL4z0CvFAxyc27GXpf02,01,3,2025,Taylor Swift,4


In [48]:
top_artists.to_sql('user_top_artists', engine, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "user_top_artists_pkey"
DETAIL:  Key (artist_id, month, user_id, year)=(3eVa5w3URK5duf6eyVDbu9, 01, 3, 2025) already exists.

[SQL: INSERT INTO user_top_artists (artist_id, month, user_id, year, artist_name, rank) VALUES (%(artist_id)s, %(month)s, %(user_id)s, %(year)s, %(artist_name)s, %(rank)s)]
[parameters: ({'artist_id': '3eVa5w3URK5duf6eyVDbu9', 'month': '01', 'user_id': 3, 'year': '2025', 'artist_name': 'ROSÉ', 'rank': 1}, {'artist_id': '57nPqD7z62gDdq37US9XJR', 'month': '01', 'user_id': 3, 'year': '2025', 'artist_name': 'Arcane', 'rank': 2}, {'artist_id': '4tuJ0bMpJh08umKkEXKUI5', 'month': '01', 'user_id': 3, 'year': '2025', 'artist_name': 'Gracie Abrams', 'rank': 3}, {'artist_id': '06HL4z0CvFAxyc27GXpf02', 'month': '01', 'user_id': 3, 'year': '2025', 'artist_name': 'Taylor Swift', 'rank': 4})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [51]:
users.iloc[0]['access_token']

'BQBH4SreJ1SwdfCDiwMg4ZO4sPjQDWPkSDA7hl9CO5BpJH9A5hZwnuB0uM-5mbE_-pFJkl4HqWjzGmv8uKEXXzbcOGqzh48S9M0J9qTqay8j1roBSHmAEDDrGLEbWABBMsu8eMIoFTnQdsqjaiRMz7NfY6kHoznH2dfQaSxak7ALlvzk8pQgNm9bqsu_A9TSTsRsWNxt9UzdemPT6Dvu8dpjtEPpilAAOflzrw'